<center> <img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>    
<table>
    <th><font size=5>Специализация "Машинное обучение и анализ данных" от:</font></th>
    <th><img src="https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://d15cw65ipctsrr.cloudfront.net/11/ae0000b18911e5965623dd71776f15/mipt.png?auto=format%2Ccompress&dpr=1&w=200&h=100&fit=clip"/> </th>
    <th><img src="https://d3njjcbhbojbot.cloudfront.net/api/utilities/v1/imageproxy/https://d15cw65ipctsrr.cloudfront.net/fe/ab6f503b2b11e796ddbfacaf40b9e6/Yandex.jpg?auto=format%2Ccompress&dpr=1&w=200&h=100&fit=clip"/></th>
</table>

# <center> ПРОЕКТ: Идентификация пользователей по посещенным веб-страницам
# <center> АВТОР ПРОЕКТА: Дмитрий Шерешевский, *PhD*
##    
<center>Исходный материал: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

<img src='http://smartcity.eletsonline.com/wp-content/uploads/2017/04/digitalindia.png'>

# <center>Часть 5 из 6.  
# <center>Соревнование Kaggle "Catch Me If You Can"
## <center>Введение

В этой части мы вспомним про концепцию стохастического градиентного спуска и опробуем классификатор Scikit-learn SGDClassifier, который работает намного быстрее на больших выборках, чем алгоритмы, которые мы тестировали в 4 части. Также мы познакомимся с данными [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) Kaggle по идентификации пользователей и сделаем в нем свои посылки. Попробуем добиться хорошего результата.

**Используемые инструменты:**
   - [Стохатический градиентный спуск](https://www.coursera.org/learn/supervised-learning/lecture/xRY50/stokhastichieskii-ghradiientnyi-spusk)
   - [Линейные модели. Sklearn.linear_model. Классификация](https://www.coursera.org/learn/supervised-learning/lecture/EBg9t/linieinyie-modieli-sklearn-linear-model-klassifikatsiia)

## <center> 5.1. Получение результата и подготовка первой посылки на Kaggle

In [1]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score
from scipy import sparse

**Считаем данные [соревнования](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [2]:
# Путь к данным
PATH_TO_DATA = ('../data')

In [3]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')

In [4]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


**Объединим обучающую и тестовую выборки – это понадобится, чтоб вместе потом привести их к разреженному формату.**

In [5]:
train_test_df = pd.concat([train_df, test_df])

В обучающей выборке видим следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - user_id – ID пользователя
    
Сессии пользователей выделены таким образом, что они не могут быть длинее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд, либо когда сессия заняла по времени более 30 минут. 

**Посмотрим на статистику признаков.**

Пропуски возникают там, где сессии короткие (менее 10 сайтов). Скажем, если человек 1 января 2015 года посетил *vk.com* в 20:01, потом *yandex.ru* в 20:29, затем *google.com* в 20:33, то первая его сессия будет состоять только из двух сайтов (site1 – ID сайта *vk.com*, time1 – 2015-01-01 20:01:00, site2 – ID сайта  *yandex.ru*, time2 – 2015-01-01 20:29:00, остальные признаки – NaN), а начиная с *google.com* пойдет новая сессия, потому что уже прошло более 30 минут с момента посещения *vk.com*.

**В обучающей выборке – 2297 сессий одного пользователя (Alice) и 251264 сессий – других пользователей, не Элис. Дисбаланс классов очень сильный, и смотреть на долю верных ответов (accuracy) непоказательно.**

In [9]:
train_df['target'].value_counts()

0    251264
1      2297
Name: target, dtype: int64

**Пока для прогноза будем использовать только индексы посещенных сайтов. Индексы нумеровались с 1, так что заменим пропуски на нули.**

In [10]:
train_test_df_sites = train_test_df[['site%d' % i for i in range(1, 11)]].fillna(0).astype(int)

In [11]:
train_test_df_sites.head(10)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0,0,0,0,0,0,0,0,0
2,890,941,3847,941,942,3846,3847,3846,1516,1518
3,14769,39,14768,14769,37,39,14768,14768,14768,14768
4,782,782,782,782,782,782,782,782,782,782
5,22,177,175,178,177,178,175,177,177,178
6,570,21,570,21,21,0,0,0,0,0
7,803,23,5956,17513,37,21,803,17514,17514,17514
8,22,21,29,5041,14422,23,21,5041,14421,14421
9,668,940,942,941,941,942,940,23,21,22


**Создадим разреженные матрицы *X_train_sparse* и *X_test_sparse* аналогично тому, как мы это делали ранее. Используем объединенную матрицу *train_test_df_sites*, потом разделим обратно на обучающую и тестовую части.**

Обратим внимание на то, что в  сессиях меньше 10 сайтов  у нас остались нули, так что первый признак (сколько раз попался 0) по смыслу отличен от остальных (сколько раз попался сайт с индексом $i$). Поэтому первый столбец разреженной матрицы надо будет удалить.

**Выделим в отдельный вектор *y* ответы на обучающей выборке.**

In [12]:
train_test_sparse = csr_matrix((np.ones(train_test_df_sites.values.size), 
                                train_test_df_sites.values.ravel(), 
                                np.arange(train_test_df_sites.values.shape[0] + 1) * 
                                train_test_df_sites.values.shape[1]), dtype=int)[:, 1:]

X_train_sparse = train_test_sparse[:train_df.shape[0]]
X_test_sparse = train_test_sparse[train_df.shape[0]:]
y = train_df['target'].values

**1. Для контроля выведем размерности матриц *X_train_sparse* и *X_test_sparse*.**

In [13]:
X_train_sparse.shape, X_test_sparse.shape

((253561, 48371), (82797, 48371))

**Сохраним в pickle-файлы объекты *X_train_sparse*, *X_test_sparse* и *y* (последний – в файл *train_target.pkl*).**

In [14]:
with open(os.path.join(PATH_TO_DATA, 'X_train_sparse.pkl'), 'wb') as X_train_sparse_pkl:
    pickle.dump(X_train_sparse, X_train_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_test_sparse.pkl'), 'wb') as X_test_sparse_pkl:
    pickle.dump(X_test_sparse, X_test_sparse_pkl, protocol=2)
with open(os.path.join(PATH_TO_DATA, 'train_target.pkl'), 'wb') as train_target_pkl:
    pickle.dump(y, train_target_pkl, protocol=2)

**Разобьем обучающую выборку на 2 части в пропорции 7/3, причем не перемешивая. Исходные данные упорядочены по времени, тестовая выборка по времени четко отделена от обучающей, это же соблюдем и здесь.**

In [15]:
train_share = int(.7 * X_train_sparse.shape[0])
X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

**Создадим объект `sklearn.linear_model.SGDClassifier` с логистической функцией потерь и параметром *random_state*=17. Остальные параметры оставим по умолчанию, разве что *n_jobs*=-1 никогда не помешает. Обучим  модель на выборке `(X_train, y_train)`.**

In [16]:
sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
sgd_logit.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=17, shuffle=True,
       tol=None, verbose=0, warm_start=False)

**Сделаем прогноз в виде предсказанных вероятностей того, что это сессия Элис, на отложенной выборке *(X_valid, y_valid)*.**

In [17]:
logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)

**2. Напечатаем ROC AUC логистической регрессии, обученной с помощью стохастического градиентного спуска, на отложенной выборке, округлив до 3 знаков после разделителя.**

In [18]:
print('ROC AUC =', round(roc_auc_score(y_valid, logit_valid_pred_proba[:, 1]), 3))

ROC AUC = 0.934


**Сделаем прогноз в виде предсказанных вероятностей отнесения к классу 1 для тестовой выборки с помощью той же *sgd_logit*, обученной уже на всей обучающей выборке (а не на 70%).**

In [19]:
%%time
sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
logit_test_pred_proba = sgd_logit.fit(X_train_sparse, y).predict_proba(X_test_sparse)[:, 1]

Wall time: 676 ms


**Запишем ответы в файл и сделайте посылку на Kaggle. Дадим своей команде (из одного человека) на Kaggle название –**    
### [YDF & MIPT] Dmitry Shereshevskiy   
**чтобы можно было легко идентифицировать посылки на [лидерборде](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard/public).**

**Результат, который мы только что получили, соответствует бейзлайну "SGDCLassifer" на лидерборде. Поставим себе задачу - максимально улучшить его.**

In [20]:
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [21]:
write_to_submission_file(logit_test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_logit.csv'))

## Пути улучшения
Что можно попробовать:
 - Использовать ранее построенные признаки для улучшения модели (проверить их можно на меньшей выборке по 150 пользователям, отделив одного из пользователей от остальных – это быстрее)
 - Настроить параметры моделей (например, коэффициенты регуляризации)
 - Если позволят мощности (или хватит терпения), можно попробовать смешивание (блендинг) ответов бустинга и линейной модели. [Вот](http://mlwave.com/kaggle-ensembling-guide/) один из самых известных тьюториалов по смешиванию ответов алгоритмов, также хороша [статья](https://alexanderdyakonov.wordpress.com/2017/03/10/cтекинг-stacking-и-блендинг-blending) Александра Дьяконова
 - Обратим внимание, что в соревновании также даны исходные данные о посещенных веб-страницах Элис и остальными 1557 пользователями (*train.zip*). По этим данным можно сформировать свою обучающую выборку. 

В 6 части мы разберем пакет **Vowpal Wabbit** и попробуем его в деле, на данных соревнования.

## <center>5.2. Работа по улучшению результата на Kaggle. Первые успехи

Сформируем новые **дополнительные признаки** из исходных даннных.

Для этого сделаем функцию, которая принимает на вход **train_df**, а выдает **X_train_sparse** с признаками-**сайтами** в виде "мешка слов" и **дополнительные признаки**, а также целевой признак **y**.

In [33]:
def feature_engineering_df(train_df, features, train=True):
    '''
    features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    '''
    # повторим, что было выше (частично)
    train_df_sites = train_df[['site%d' % i for i in range(1, 11)]] # не забыть заполнить нули
    
    # подготовим временнЫе метки
    train_df[['time%d' % i for i in range(1, 11)]] = train_df[['time%d' % i for i in range(1, 11)]].applymap(pd.to_datetime)
    timestamp_df = train_df[['time%d' % i for i in range(1, 11)]].applymap(lambda x: pd.datetime.timestamp(x) 
                                                                  if type(x) == pd._libs.tslib.Timestamp else np.nan)
    # добавляем признаки
    if '#unique_sites' in features:
        train_df['#unique_sites'] = train_df[['site' + str(i) for i in range(1, 11)]].nunique(axis=1, dropna=True)
    if 'session_timespan' in features:
        train_df['session_timespan'] = timestamp_df.max(axis=1, skipna=True) - timestamp_df.min(axis=1, skipna=True)
    if 'start_hour' in features:    
        train_df['start_hour'] = train_df['time1'].apply(lambda x: x.hour)
    if 'day_of_week' in features:    
        train_df['day_of_week'] = train_df['time1'].apply(lambda x: x.weekday())
    if 'time_of_day' in features:    
        # время суток (06-12 - утро(1), 12-19 - день(2), 19-24 - вечер(3), 00-06 - ночь(4))
        train_df['time_of_day'] = train_df['time1'].apply(lambda x:  1 if x.hour >  6 and x.hour <= 12 else 
                                                                     2 if x.hour > 12 and x.hour <= 19 else
                                                                     3 if x.hour > 19 and x.hour <= 24 else 
                                                                     4)
    if 'weekend' in features:
        # индикатор уикэнда в день начала сессии
        train_df['weekend'] = train_df['time1'].apply(lambda x: 1 if x.weekday() in [5, 6] else 0)
   
    train_df_sites = train_df_sites.fillna(0).astype('int')
    train_sparse = csr_matrix((np.ones(train_df_sites.values.size), 
                                train_df_sites.values.ravel(), 
                                np.arange(train_df_sites.values.shape[0] + 1) * 
                                train_df_sites.values.shape[1]), dtype=int)[:, 1:]
    if train:
        y = train_df['target'].values
    else:
        y = None

    return train_sparse, train_df[features], y

Для автоматизации исследования признаков также **создадим функцию**, которая принимает на вход размеченные данные в виде **матрицы признаков** и вектора **целевой функции**, а на выходе дает метрику **roc_auc** по отложенной выборке размером **30%** от исходных данных.   
Вообще-то это не совсем корректно и правильнее использовать для кнтроля метрики **кросс-валидацию**, но на первом этапе для "прикидок" этого, скорее всего, будет достаточно, а аналогичную функцию с **кросс-валидацией** сделаем позже.

In [23]:
def logit_roc_auc_score(X_train_sparse, y):
    '''
    обучение и расчет метрики на отложенной выборке
    '''

    train_share = int(.7 * X_train_sparse.shape[0])
    X_train, y_train = X_train_sparse[:train_share, :], y[:train_share]
    X_valid, y_valid  = X_train_sparse[train_share:, :], y[train_share:]

    sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
    sgd_logit.fit(X_train, y_train)

    logit_valid_pred_proba = sgd_logit.predict_proba(X_valid)

    return roc_auc_score(y_valid, logit_valid_pred_proba[:, 1])

Проверим **все** комбинации добавления признаков:

In [25]:
import itertools

In [35]:
features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
train_sites_sparse, features_df, y_values = feature_engineering_df(train_df, features, train=True)

feature_scores = []
for r in range(len(features) + 1):
    for fe in itertools.combinations(features, r):
        fe_ = list(fe)
        features_sparse = csr_matrix(features_df[fe_].values)
  
        X_train_sparse = sparse.hstack([train_sites_sparse , features_sparse]).tocsr()
        
        logit_roc_auc = logit_roc_auc_score(X_train_sparse, y_values)
        feature_scores.append((logit_roc_auc, fe_))

sorted_features = sorted(feature_scores, key=lambda x: x[0], reverse=True)

print('5 лучших метрик с фичами')
for pair in sorted_features[:5]:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

print('\n5 худших метрик с фичами')
for pair in sorted_features[-5:]:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

5 лучших метрик с фичами
roc_auc_score: 0.934381940306 features: ['weekend']
roc_auc_score: 0.933847286791 features: []
roc_auc_score: 0.924980978083 features: ['day_of_week', 'weekend']
roc_auc_score: 0.924717063844 features: ['day_of_week']
roc_auc_score: 0.923196690838 features: ['#unique_sites', 'weekend']

5 худших метрик с фичами
roc_auc_score: 0.674627312681 features: ['session_timespan', 'day_of_week', 'weekend']
roc_auc_score: 0.668984875716 features: ['session_timespan', 'weekend']
roc_auc_score: 0.66837137001 features: ['session_timespan', 'time_of_day', 'weekend']
roc_auc_score: 0.666589603811 features: ['session_timespan', 'time_of_day']
roc_auc_score: 0.665770081657 features: ['session_timespan']


**ВЫВОД:**    
Как показал этот анализ, только добавление **одного** из всех признаков - а именно **'weekend'** - дало **увеличение** метрики **ROC AUC**.
Остальные признаки только лишь **уменьшили** метрику **ROC AUC**.   
Причем особенно **негативно** влияет присутствие признака **'session_timespan'**.  

На первый взгляд результат может **показаться страным** - ведь интуитивно кажется, что **добавление новых признаков** должно результат **улучшать**.   
Но если посмотреть на признаки повнимательнее, то можно увидеть, что дополнительные признаки, являясь по форме **числовыми и количественными**, по сути же, если разобраться в их природе, являются **категориальными**. Поэтому обращение с ними как с **числовыми признаками**, имеющими отношение **порядка**, и привело к ухудшению результата. 

Для корректного их использования каждый из них нужно **бинаризовать**. Как это правильно сделать мы поговорим ниже. Здесь же отметим, что именно признак **weekend** по своей природе уже и так **является бинарным**, поскольку принимает только 2 значения - *0* и *1*, и именно поэтому его использование в таком виде **уже** является корректным и приводит к **улучшению** результата.    

Использование же других дополнительных признаков стало **демонстрацией некорректного применения** признаков. Эту ситуацию мы исправим ниже.
#### ИТАК, на этом этапе добавляем только признак **'weekend'**.

Добавим  **'weekend'**, подготовим данные и сформируем посылку на **Kaggle**.

In [51]:
%%time
train_sites_sparse, features_df, y_train = feature_engineering_df(train_test_df, features=features, train=True)
features_sparse = csr_matrix(features_df[['weekend']].values)
X_train_test_sparse_newfe = sparse.hstack([train_sites_sparse , features_sparse]).tocsr()

# делим на трейн и тест
X_train_sparse_newfe = X_train_test_sparse_newfe[:train_df.shape[0]]
X_test_sparse_newfe = X_train_test_sparse_newfe[train_df.shape[0]:]
y_train = y_train[:train_df.shape[0]]

Wall time: 6min 25s


In [52]:
%%time
sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
logit_test_pred_proba = sgd_logit.fit(X_train_sparse_newfe, y_train).predict_proba(X_test_sparse_newfe)[:, 1]
    
write_to_submission_file(logit_test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_logit_newfe.csv'))

Wall time: 1.22 s


Сделаем соответствующую **посылку на Kaggle**.   
Её результат:   
- **+46** позиций вверх    
- Score Public Leaderboard = **0.92600**   
- Бенчмарк "sgd_logit_benchmark.csv" на публичной части рейтинга в соревновании Kaggle - **побит**.

## <center> 5.3. Настраиваем параметры и пробуем упрощенную предобработку признаков

В данном соревновании **длина сессии** и **ширина окна** заданы жестко как **(10, 10)**, поэтому подобирать эти параметры не получается.

В связи с этим будем работать с настройками SGDClassifier:
- альфа
- балансировка классов

Для оптимизации временн**ы**х затрат поработаем с выборкой на 150 пользователей.

Загрузим сохраненные ранее данные и сформируем выборку для работы.

In [53]:
with open(os.path.join(PATH_TO_DATA, 'selected_features_150users.pkl'), 'rb') as selected_features_150users_pkl:
    selected_features_150users = pickle.load(selected_features_150users_pkl)
    
with open(os.path.join(PATH_TO_DATA, 'new_features_150users.pkl'), 'rb') as new_features_150users_pkl:
    new_features_150users = pickle.load(new_features_150users_pkl)
    
with open(os.path.join(PATH_TO_DATA, 'X_sparse_150users.pkl'), 'rb') as X_sparse_150users_pkl:
    X_sparse_150users = pickle.load(X_sparse_150users_pkl)
    
with open(os.path.join(PATH_TO_DATA, 'y_150users.pkl'), 'rb') as y_150users_pkl:
    y_150users = pickle.load(y_150users_pkl)

In [54]:
selected_features_150users.head()

,start_year,start_month,start_day,time_of_day,freq_top30_0_1,freq_top30_1_3,freq_top30_2_41,freq_top30_3_2,freq_top30_4_181,freq_top30_5_214,...,freq_top30_21_1201,freq_top30_22_6,freq_top30_23_2165,freq_top30_24_244,freq_top30_25_180,freq_top30_26_1463,freq_top30_27_106,freq_top30_28_253,freq_top30_29_2707,weekend
0,2013,11,15,1,4,2,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2013,11,15,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2013,11,15,1,5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2013,11,15,1,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2013,11,15,1,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
selected_features_150users.shape, y_150users.shape

((137019, 35), (137019,))

In [56]:
X_sparse_150users

<137019x27797 sparse matrix of type '<class 'numpy.int32'>'
	with 1369510 stored elements in Compressed Sparse Row format>

Возьмем отобранный выше признак **'weekend'** и добавим к сформированному ранее **"мешку слов"** с сайтами. Скорректируем также целевой признак под **бинарную** классификацию.

In [57]:
X_150_newfe_sparse = csr_matrix(selected_features_150users[['weekend']].values)
  
X_150_train_newfe_sparse = sparse.hstack([X_sparse_150users, X_150_newfe_sparse]).tocsr()

In [58]:
y_150_train = (y_150users == 128).astype(int)

Для подбора параметров воспльзуемся функцией **GridSearchCV** из библиотеки `sklearn`:

In [59]:
%%time
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, random_state=17)

sgd_params = {'alpha': np.logspace(-8, 1, 40)}

sgd_grid_searcher = GridSearchCV(sgd_logit, param_grid=sgd_params, scoring='roc_auc', n_jobs=-1, cv=skf)
sgd_grid_searcher.fit(X_150_train_newfe_sparse, y_150_train)

Wall time: 39.3 s


In [60]:
print(sgd_grid_searcher.best_params_, sgd_grid_searcher.best_score_)

{'alpha': 4.9238826317067415e-05} 0.971957399925


Попробуем тот же набор параметров, но **добавим балансировку классов** в **SGDClassifier()**, что логично, поскольку, как мы видели в 4 части, в исследуемой выборке есть большой дисбаланс классов.

In [61]:
%%time
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)

sgd_params = {'alpha': np.logspace(-8, 1, 40)}

sgd_grid_searcher = GridSearchCV(sgd_logit, param_grid=sgd_params, scoring='roc_auc', n_jobs=-1, cv=skf)
sgd_grid_searcher.fit(X_150_train_newfe_sparse, y_150_train)

Wall time: 37.5 s


In [62]:
print(sgd_grid_searcher.best_params_, sgd_grid_searcher.best_score_)

{'alpha': 0.00041246263829013564} 0.977036274013


**ВЫВОД: **   
результат, как и ожидалось, **улучшился**.    
Обучим на этих параметрах классификатор и сформируем посылку на **Kaggle**

In [63]:
%%time
# train_sites_sparse, features_df, y_train = feature_engineering_df(train_test_df, features=features, train=True)
# features_sparse = csr_matrix(features_df[['weekend']].values)
# X_train_test_sparse_newfe = sparse.hstack([train_sites_sparse , features_sparse]).tocsr()

# делим на трейн и тест
X_train_sparse_newfe = X_train_test_sparse_newfe[:train_df.shape[0]]
X_test_sparse_newfe = X_train_test_sparse_newfe[train_df.shape[0]:]
y_train = y_train[:train_df.shape[0]]

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, alpha=sgd_grid_searcher.best_params_['alpha'], 
                          class_weight='balanced', random_state=17)
logit_test_pred_proba = sgd_logit.fit(X_train_sparse_newfe, y_train).predict_proba(X_test_sparse_newfe)[:, 1]
    
write_to_submission_file(logit_test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_logit_newfe_alpha_bal.csv'))

Wall time: 1.31 s


Против ожиданий, при таких параметрах **auc_roc** на **Kaggle** больше не стал, наоборот - **уменьшился**: **0.92275** (было 0.92600).    
Будем подбирать дальше.

**ПРОМЕЖУТОЧНЫЙ ВЫВОД:**    
Пока подбор параметра **alpha** по выборке на 150 пользователях положительного результата не дал.    
Возможно, это связано с небольшим отличием правила формирования тренировочной выборки, на которой проводился подбор, от правила для выборки, которая участвует в соревновании (исходные данные одинаковы).

#### Поработаем еще с признаками.
Для этого сформируем (как планировали выше) дополнительную функцию, которая оценивает оптимизируемую метрику уже **не по отложенной выборке**, а с использованием процедуры **кросс-валидации**.

In [64]:
def CV_score(estimator, X_train_sparse, y, cv=None, random_state=17, test_size=0.3):
    '''
    обучение и расчет метрики на отложенной выборке
    '''
    # Разобьем выборку на 2 части. Учтем, что разбиение выборки с train_test_split должно быть стратифицированным
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse, y, 
                                                          test_size=test_size, 
                                                          random_state=random_state, stratify=y)
    mean_CV_score = cross_val_score(estimator, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1).mean()
 
    return mean_CV_score

**Далее, теперь будем учитывать обстоятельство, что большинство дополнительных фич у нас по сути КАТЕГОРИАЛЬНЫЕ, и для них нужно делать определенную предобработку**.
Мы уже обсуждали это ранее. Как мы отмечали, для корректности категориальные признаки надо **бинаризовать**. Более правильно это сделать с применением техники **'one hot coding'** или  **Tf-idf**-кодирования. Сделаем это ниже, а пока попробуем **упрощенный вариант** - просто поделим пространства значений признаков на 2 части и примем значение признака равно **1**, если он принадлежит к одной из половин, и **0**, если он к ней не принадлежит.

В соответствии с этими соображениями модифицируем функцию для подготовки признаков для такой **упрощенной** бинаризации.

In [261]:
def feature_engineering_df_bin(train_df, features=None, train=True):
    '''
    features = ['#unique_sites_bin', 'session_timespan_bin', 'time_of_day', 'weekend']
    '''
    
    if features == None:
        features_ = ['#unique_sites_bin', 'session_timespan_bin', 'time_of_day', 'weekend']
    else:
        features_ = features
    
    # готовим сайты
    train_df_sites = train_df[['site%d' % i for i in range(1, 11)]] # наны заполним нулями потом
    
    # подготовим временнЫе метки
    train_df[['time%d' % i for i in range(1, 11)]] = train_df[['time%d' % i for i in range(1, 11)]].applymap(pd.to_datetime)
    timestamp_df = train_df[['time%d' % i for i in range(1, 11)]].applymap(lambda x: pd.datetime.timestamp(x) 
                                                                  if type(x) == pd._libs.tslib.Timestamp else np.nan)
    # добавляем признаки
    if '#unique_sites_bin' in features_:
        train_df['#unique_sites_bin'] = (train_df[['site' + str(i) for i in range(1, 11)]].nunique(axis=1, dropna=True)). \
                                         apply(lambda x: 1 if x < 6 else 0)
    if 'session_timespan_bin' in features_:
        train_df['session_timespan_bin'] = (timestamp_df.max(axis=1, skipna=True) - timestamp_df.min(axis=1, skipna=True)). \
                                            apply(lambda x: 1 if x < 50 else 0)
#     if 'start_hour' in features:    
#         train_df['start_hour'] = train_df['time1'].apply(lambda x: x.hour)
#     if 'day_of_week' in features:    
#         train_df['day_of_week'] = train_df['time1'].apply(lambda x: x.weekday())
    if 'time_of_day' in features_:    
        # до обеда - после обеда
        train_df['time_of_day'] = train_df['time1'].apply(lambda x:  1 if x.hour <  14 else 0)
    if 'weekend' in features_:
        # индикатор уикэнда в день начала сессии
        train_df['weekend'] = train_df['time1'].apply(lambda x: 1 if x.weekday() in [5, 6] else 0)
 
    train_df_sites = train_df_sites.fillna(0).astype('int')
    train_sparse = csr_matrix((np.ones(train_df_sites.values.size), 
                                train_df_sites.values.ravel(), 
                                np.arange(train_df_sites.values.shape[0] + 1) * 
                                train_df_sites.values.shape[1]), dtype=int)[:, 1:]
    if train:
        y = train_df['target'].values
    else:
        y = None

    return train_sparse, train_df[features_], y

In [250]:
%%time
train_sites_sparse, selected_features_df, y_values = feature_engineering_df_bin(train_df, train=True)

Wall time: 53.5 s


In [251]:
%%time
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
features = selected_features_df.columns
feature_scores = []
for r in range(len(features) + 1):
    for fe in itertools.combinations(features, r):
        fe_ = list(fe)
        selected_features_sparse = csr_matrix(selected_features_df[fe_].values)
  
        X_train_new_sparse = sparse.hstack([train_sites_sparse , selected_features_sparse]).tocsr()
        
        estimator = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
        roc_auc_score = CV_score(estimator, X_train_new_sparse, y_values, cv=skf, 
                                 random_state=17, test_size=0.3)
        feature_scores.append((roc_auc_score, fe_))

sorted_features = sorted(feature_scores, key=lambda x: x[0], reverse=True)

print('Без баланса')
print('\n5 лучших метрик с фичами')
for pair in sorted_features[:5]:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

print('\n5 худших метрик с фичами')
for pair in sorted_features[-5:]:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

Без баланса

5 лучших метрик с фичами
roc_auc_score: 0.934218622552 features: ['weekend']
roc_auc_score: 0.933635284594 features: ['time_of_day', 'weekend']
roc_auc_score: 0.932279637877 features: []
roc_auc_score: 0.930773100919 features: ['time_of_day']
roc_auc_score: 0.929718057263 features: ['#unique_sites_bin', 'time_of_day', 'weekend']

5 худших метрик с фичами
roc_auc_score: 0.923128645975 features: ['session_timespan_bin', 'weekend']
roc_auc_score: 0.920592024569 features: ['#unique_sites_bin', 'session_timespan_bin', 'time_of_day']
roc_auc_score: 0.920483235092 features: ['session_timespan_bin']
roc_auc_score: 0.915035943593 features: ['#unique_sites_bin', 'session_timespan_bin', 'weekend']
roc_auc_score: 0.910999149669 features: ['#unique_sites_bin', 'session_timespan_bin']
Wall time: 5min 30s


#### Проделаем то же самое, делая баланс в обучающей выборке

In [253]:
%%time
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
features = selected_features_df.columns
feature_scores = []
for r in range(len(features) + 1):
    for fe in itertools.combinations(features, r):
        fe_ = list(fe)
        selected_features_sparse = csr_matrix(selected_features_df[fe_].values)
  
        X_train_new_sparse = sparse.hstack([train_sites_sparse , selected_features_sparse]).tocsr()
        
        estimator = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
        roc_auc_score = CV_score(estimator, X_train_new_sparse, y_values, cv=skf, 
                                 random_state=17, test_size=0.3)
        feature_scores.append((roc_auc_score, fe_))

sorted_features = sorted(feature_scores, key=lambda x: x[0], reverse=True)

print('С балансом')
print('\n5 лучших метрик с фичами')
for pair in sorted_features[:5]:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

print('\n5 худших метрик с фичами')
for pair in sorted_features[-5:]:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

С балансом

5 лучших метрик с фичами
roc_auc_score: 0.938555892249 features: ['#unique_sites_bin', 'session_timespan_bin', 'time_of_day', 'weekend']
roc_auc_score: 0.936702678996 features: ['#unique_sites_bin', 'session_timespan_bin', 'time_of_day']
roc_auc_score: 0.936023752468 features: ['#unique_sites_bin', 'session_timespan_bin', 'weekend']
roc_auc_score: 0.935668706936 features: ['session_timespan_bin', 'time_of_day', 'weekend']
roc_auc_score: 0.934530145929 features: ['#unique_sites_bin', 'session_timespan_bin']

5 худших метрик с фичами
roc_auc_score: 0.930224205183 features: ['weekend']
roc_auc_score: 0.930118730518 features: ['session_timespan_bin']
roc_auc_score: 0.929995398298 features: ['#unique_sites_bin']
roc_auc_score: 0.929944599468 features: ['time_of_day']
roc_auc_score: 0.926259578693 features: []
Wall time: 4min 52s


Видим, что **с балансом** результат улучшился.    
Возьмем комбинацию лучших фич и на ее основе сформируем посылку на **Kaggle**.

In [256]:
sorted_features[0][1]

['#unique_sites_bin', 'session_timespan_bin', 'time_of_day', 'weekend']

In [262]:
%%time
features = ['#unique_sites_bin', 'session_timespan_bin', 'time_of_day', 'weekend']
train_sites_sparse, selected_features_df, y_values
X_train_test_sites_sparse, selected_train_test_features_df, y_train_none = \
                                                        feature_engineering_df_bin(train_test_df, features=features, train=True)
# делим на трейн и тест
selected_train_test_features_sparse = csr_matrix(selected_train_test_features_df[features].values)
  
X_train_test_newfe_sparse = sparse.hstack([X_train_test_sites_sparse , selected_train_test_features_sparse]).tocsr()
X_train_newfe_sparse = X_train_test_newfe_sparse[:train_df.shape[0]]
X_test_newfe_sparse = X_train_test_newfe_sparse[train_df.shape[0]:]
y_train = y_train_none[:train_df.shape[0]]

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
logit_test_pred_proba = sgd_logit.fit(X_train_newfe_sparse, y_train).predict_proba(X_test_newfe_sparse)[:, 1]
    
write_to_submission_file(logit_test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_logit_4features_bal.csv'))

Wall time: 1min 52s


Получили на **Kaggle: 0.90482**.    
Видим, что такая **упрощенная** предобработка категориальных признаков хорошего результата не дала.

### Подведем промежуточный итог
- правильно предобработанные категориальные признаки улучшают результат предсказания
- балансировка выборки по целевому признаку результат улучшает
- упрощенная предобработка категориальных признаков не дает хорошего результата
- добавление некорректно предобработанных категориальных признаков результат **ухудшает**

## <center> 5.4. Пробуем БУСТИНГ

### Рассмотрим возможность применить CatBoostClassifier
```python
%%time
# пробуем CatBoostClassifier
import catboost
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
features = ['time_of_day', 'weekend']
feature_scores = []
for r in range(len(features) + 1):
    for fe in itertools.combinations(features, r):
        fe_ = list(fe)
        selected_features_sparse = csr_matrix(selected_features_df[fe_].values)
  
        X_train_new_sparse = sparse.hstack([train_sites_sparse , selected_features_sparse]).tocsr()
        
        estimator = catboost.CatBoostClassifier()
        roc_auc_score = CV_score(estimator, X_train_new_sparse.toarray(), y_values, cv=skf, 
                                 random_state=17, test_size=0.3)
        feature_scores.append((roc_auc_score, fe_))

sorted_features = sorted(feature_scores, key=lambda x: x[0], reverse=True)

for pair in sorted_features:
    print('roc_auc_score:', pair[0], 'features:', pair[1])
    
```
**РЕЗЮМЕ:**    
Catboost **не поддерживае**т работу с **sparse**-форматом, поэтому пока не подходит для этой задачи (а перевод в **np.array** приводит к **ошибке памяти**, скорее всего - памяти для этого просто не хватает)

### Пробуем XGBOOST, поскольку он умеет работать со sparse-форматом

In [282]:
import xgboost as xgb

In [272]:
%%time
# пробуем xgboost
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)
features = ['time_of_day', 'weekend']
feature_scores = []
for r in range(len(features) + 1):
    for fe in itertools.combinations(features, r):
        fe_ = list(fe)
        selected_features_sparse = csr_matrix(selected_features_df[fe_].values)
  
        X_train_new_sparse = sparse.hstack([train_sites_sparse , selected_features_sparse]).tocsr()
        
        estimator = xgb.XGBClassifier(n_jobs=-1, random_state=17)
        roc_auc_score = CV_score(estimator, X_train_new_sparse, y_values, cv=skf, 
                                 random_state=17, test_size=0.3)
        feature_scores.append((roc_auc_score, fe_))

sorted_features = sorted(feature_scores, key=lambda x: x[0], reverse=True)

for pair in sorted_features:
    print('roc_auc_score:', pair[0], 'features:', pair[1])

roc_auc_score: 0.904199430799 features: ['time_of_day', 'weekend']
roc_auc_score: 0.899112212739 features: ['time_of_day']
roc_auc_score: 0.890882452164 features: ['weekend']
roc_auc_score: 0.884568244521 features: []
Wall time: 5min 4s


Результат пока **не очень хороший**.   
Модифицируем **feature_engineering_df**, изменив **session_timespan** таким образом, чтобы он принимал только **7** значений:

In [305]:
def feature_engineering_df_mod(train_df, test_df=None, features=None):
    '''
    features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    '''
    if features is None:
        features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    
    if test_df is None:
        df = train_df
    else:
        df = pd.concat([train_df, test_df])
    
    # выделим отдельно сайты
    df_sites = df[['site%d' % i for i in range(1, 11)]] # не забыть заполнить нули
    
    # подготовим временнЫе метки
    df[['time%d' % i for i in range(1, 11)]] = df[['time%d' % i for i in range(1, 11)]].applymap(pd.to_datetime)
    timestamp_df = df[['time%d' % i for i in range(1, 11)]].applymap(lambda x: pd.datetime.timestamp(x) 
                                                                  if type(x) == pd._libs.tslib.Timestamp else np.nan)
    # добавляем признаки
    if '#unique_sites' in features:
        df['#unique_sites'] = df[['site' + str(i) for i in range(1, 11)]].nunique(axis=1, dropna=True)
    if 'session_timespan' in features:
        df['session_timespan'] = timestamp_df.max(axis=1, skipna=True) - timestamp_df.min(axis=1, skipna=True)
        df['session_timespan'] = df['session_timespan'].apply(lambda x: x // 30 if x < 210 else 7)
    if 'start_hour' in features:    
        df['start_hour'] = df['time1'].apply(lambda x: x.hour)
    if 'day_of_week' in features:    
        df['day_of_week'] = df['time1'].apply(lambda x: x.weekday())
    if 'time_of_day' in features:    
        # время суток (06-12 - утро(1), 12-19 - день(2), 19-24 - вечер(3), 00-06 - ночь(4))
        df['time_of_day'] = df['time1'].apply(lambda x:  1 if x.hour >  6 and x.hour <= 12 else 
                                                                     2 if x.hour > 12 and x.hour <= 19 else
                                                                     3 if x.hour > 19 and x.hour <= 24 else 
                                                                     4)
    if 'weekend' in features:
        # индикатор уикэнда в день начала сессии
        df['weekend'] = df['time1'].apply(lambda x: 1 if x.weekday() in [5, 6] else 0)
    
    X_new_sparse = csr_matrix(df[features].values)
    
    df_sites = df_sites.fillna(0).astype('int')
    train_sparse = csr_matrix((np.ones(df_sites.values.size), 
                                df_sites.values.ravel(), 
                                np.arange(df_sites.values.shape[0] + 1) * 
                                df_sites.values.shape[1]), dtype=int)[:, 1:]
    
    
    y_train = df['target'].values[:train_df.shape[0]]
        
    X_sparse = sparse.hstack([train_sparse, X_new_sparse]).tocsr()
    X_train_sparse = X_sparse[:train_df.shape[0]]
    X_test_sparse = X_sparse[train_df.shape[0]:]
    
    
    
    return (X_train_sparse, y_train) if test_df is None else (X_train_sparse, y_train, X_test_sparse)

In [291]:
%%time
# готовим данные
X_train_new_sparse, y_train = feature_engineering_df_mod(train_df)

Wall time: 1min 26s


In [293]:
# инициализируем параметры
params = {
    'objective': 'binary:logistic',
    'max_depth': 3,
    'learning_rate': 0.5,
    'silent': 1.0,
    'n_estimators': 170
}

In [314]:
estimator = xgb.XGBClassifier(n_jobs=-1, random_state=17, **params)
roc_auc_score = CV_score(estimator, X_train_new_sparse, y_train, 
                         random_state=17, test_size=0.3)
roc_auc_score

0.98638590813925198

Вот теперь **roc_auc** получился довольно приличный!    
Подготовим и сделаем на этих параметрах посылку на **Kaggle**

In [306]:
%%time
X_train_sparse, y_train, X_test_sparse = feature_engineering_df_mod(train_df, test_df)

Wall time: 2min 50s


In [307]:
%%time
params = {
    'objective': 'binary:logistic',
    'max_depth': 3,
    'learning_rate': 0.5,
    'silent': 1.0,
    'n_estimators': 170
}


estimator = xgb.XGBClassifier(n_jobs=-1, random_state=17, **params)
test_pred_proba = estimator.fit(X_train_sparse, y_train).predict_proba(X_test_sparse)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_xgb_allfeatures_1.csv'))

Wall time: 28.8 s


**Kaggle: 0.90088**     
Против ожидания по итогам предварительной метрики, результат **на Kaggle не высокий**.    
С другой стороны, это **согласуется** с отмеченным ранее обстоятельством, что для **подобного рода задач** лучше подходят именно **линейные** методы. Так что все в пределах разумного.

**Промежуточное РЕЗЮМЕ:**   
Видим, что **xgboost с логисической регрессией**, несмотря на **высокие результаты на кроссвалидации**, на **Kaggle** хорошего результата **не дает**.    

Будем иметь ввиду бустинг в дальнейшем для **блендинга** - смешивания различных алгоритмов. А пока снова вернемся к линейным методам. 

## <center> 5.5. Предобработка категориальных признаков
**Вернемся к SGDClassifier**.    
Сделаем предобработку категориальных признаков по алгоритму **One Hot Encoding (OHE)** и применим **SGDClassifier**, с которым в предыдущих разделах получился **достаточно высокий AUC**.

In [18]:
from sklearn.preprocessing import OneHotEncoder

Модифицируем подготовку признаков, добавив **OHE**

In [319]:
def feature_engineering_df_OHE(train_df, test_df=None, features=None):
    '''
    features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    '''
    if features is None:
        features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    
    if test_df is None:
        df = train_df
    else:
        df = pd.concat([train_df, test_df])
    
    # выделим отдельно сайты
    df_sites = df[['site%d' % i for i in range(1, 11)]] # не забыть заполнить нули
    
    # подготовим временнЫе метки
    df[['time%d' % i for i in range(1, 11)]] = df[['time%d' % i for i in range(1, 11)]].applymap(pd.to_datetime)
    timestamp_df = df[['time%d' % i for i in range(1, 11)]].applymap(lambda x: pd.datetime.timestamp(x) 
                                                                  if type(x) == pd._libs.tslib.Timestamp else np.nan)
    # добавляем признаки
    if '#unique_sites' in features:
        df['#unique_sites'] = df[['site' + str(i) for i in range(1, 11)]].nunique(axis=1, dropna=True)
    if 'session_timespan' in features:
        df['session_timespan'] = timestamp_df.max(axis=1, skipna=True) - timestamp_df.min(axis=1, skipna=True)
        df['session_timespan'] = df['session_timespan'].apply(lambda x: x // 30 if x < 210 else 7)
    if 'start_hour' in features:    
        df['start_hour'] = df['time1'].apply(lambda x: x.hour)
    if 'day_of_week' in features:    
        df['day_of_week'] = df['time1'].apply(lambda x: x.weekday())
    if 'time_of_day' in features:    
        # время суток (06-12 - утро(1), 12-19 - день(2), 19-24 - вечер(3), 00-06 - ночь(4))
        df['time_of_day'] = df['time1'].apply(lambda x:  1 if x.hour >  6 and x.hour <= 12 else 
                                                                     2 if x.hour > 12 and x.hour <= 19 else
                                                                     3 if x.hour > 19 and x.hour <= 24 else 
                                                                     4)
    if 'weekend' in features:
        # индикатор уикэнда в день начала сессии
        df['weekend'] = df['time1'].apply(lambda x: 1 if x.weekday() in [5, 6] else 0)
    
    encoder = OneHotEncoder()
    #X_new_sparse = csr_matrix(df[features].values)
    X_new_sparse = encoder.fit_transform(df[features].values)
    
    df_sites = df_sites.fillna(0).astype('int')
    train_sparse = csr_matrix((np.ones(df_sites.values.size), 
                                df_sites.values.ravel(), 
                                np.arange(df_sites.values.shape[0] + 1) * 
                                df_sites.values.shape[1]), dtype=int)[:, 1:]
    
    
    y_train = df['target'].values[:train_df.shape[0]]
        
    X_sparse = sparse.hstack([train_sparse, X_new_sparse]).tocsr()
    X_train_sparse = X_sparse[:train_df.shape[0]]
    X_test_sparse = X_sparse[train_df.shape[0]:]
    
    
    
    return (X_train_sparse, y_train) if test_df is None else (X_train_sparse, y_train, X_test_sparse)

In [321]:
%%time
X_train_sparse_ohe, y_train_ohe, X_test_sparse_ohe = feature_engineering_df_OHE(train_df, test_df)

Wall time: 3min 6s


In [323]:
%%time
estimator = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
auc = CV_score(estimator, X_train_sparse_ohe, y_train_ohe, cv=skf)
print(auc)

0.973683288799
Wall time: 37.7 s


Результат впечатляет, попробуем на **Kaggle**

In [324]:
%%time
estimator = SGDClassifier(loss='log', n_jobs=-1, random_state=17)
test_pred_proba = estimator.fit(X_train_sparse_ohe, y_train_ohe).predict_proba(X_test_sparse_ohe)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_allfeatures_OHE.csv'))

Wall time: 1.7 s


**Kaggle: 0.93742**    
Наконец то!    
:))   
 Результат достаточно высокий. Пробуем его еще улучшить.

Сделаем с **балансировкой** классов.

In [325]:
%%time
estimator = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
auc = CV_score(estimator, X_train_sparse_ohe, y_train_ohe, cv=skf)
print(auc)

0.977325731478
Wall time: 18.6 s


Результат улучшился. Сделаем посылку на Kaggle

In [326]:
%%time
estimator = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
test_pred_proba = estimator.fit(X_train_sparse_ohe, y_train_ohe).predict_proba(X_test_sparse_ohe)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_allfeatures_OHE_bal.csv'))

Wall time: 1.45 s


**Kaggle: 0.94607** - результат **еще более улучшился!**   
:)))

Сделаем подбор **alpha** в SGDClassifier.

In [328]:
%%time
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=17)

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)

sgd_params = {'alpha': np.logspace(-8, 1, 40)}

sgd_grid_searcher = GridSearchCV(sgd_logit, param_grid=sgd_params, scoring='roc_auc', n_jobs=-1, cv=skf)
sgd_grid_searcher.fit(X_train_sparse_ohe, y_train_ohe)

print(sgd_grid_searcher.best_params_, sgd_grid_searcher.best_score_)

{'alpha': 0.00041246263829013564} 0.981989687948
Wall time: 1min 3s


In [329]:
%%time
estimator = SGDClassifier(loss='log', n_jobs=-1, alpha=sgd_grid_searcher.best_params_['alpha'], 
                          class_weight='balanced', random_state=17)
test_pred_proba = estimator.fit(X_train_sparse_ohe, y_train_ohe).predict_proba(X_test_sparse_ohe)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_allfeatures_OHE_bal_alpha.csv'))

Wall time: 1.38 s


**Kaggle: 0.94478 - немного уменьшился**.   

Видим, что подбор **alpha** пока результата на **Kaggle** не дал: несмотря на **увеличение метрики на кроссвалидации**, на Leader Board **результат ухудшился**.  

Попробуем для предобработки категориальных признаков алгоритм **TfidfVectorizer()**, который более гибко учитывает частоту появления тех или иных значений признака.

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
def feature_engineering_df_OHE_Tfidf(train_df, test_df=None, features=None):
    '''
    features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    '''
    if features is None:
        features = ['#unique_sites', 'session_timespan', 'start_hour', 'day_of_week', 'time_of_day', 'weekend']
    
    if test_df is None:
        df = train_df
    else:
        df = pd.concat([train_df, test_df])
    
    # выделим отдельно сайты
    df_sites = df[['site%d' % i for i in range(1, 11)]] # не забыть заполнить нули
    
    # подготовим временнЫе метки
    df[['time%d' % i for i in range(1, 11)]] = df[['time%d' % i for i in range(1, 11)]].applymap(pd.to_datetime)
    timestamp_df = df[['time%d' % i for i in range(1, 11)]].applymap(lambda x: pd.datetime.timestamp(x) 
                                                                  if type(x) == pd._libs.tslib.Timestamp else np.nan)
    # добавляем признаки
    if '#unique_sites' in features:
        df['#unique_sites'] = df[['site' + str(i) for i in range(1, 11)]].nunique(axis=1, dropna=True)
    if 'session_timespan' in features:
        df['session_timespan'] = timestamp_df.max(axis=1, skipna=True) - timestamp_df.min(axis=1, skipna=True)
        df['session_timespan'] = df['session_timespan'].apply(lambda x: x // 30 if x < 210 else 7)
#         df['session_timespan'] = df['session_timespan'].apply(lambda x: int(np.log2(x)) if x > 1 else 0)
    if 'start_hour' in features:    
        df['start_hour'] = df['time1'].apply(lambda x: x.hour)
    if 'day_of_week' in features:    
        df['day_of_week'] = df['time1'].apply(lambda x: x.weekday())
    if 'time_of_day' in features:    
        # время суток (06-12 - утро(1), 12-19 - день(2), 19-24 - вечер(3), 00-06 - ночь(4))
        df['time_of_day'] = df['time1'].apply(lambda x:  1 if x.hour >  6 and x.hour <= 12 else 
                                                                     2 if x.hour > 12 and x.hour <= 19 else
                                                                     3 if x.hour > 19 and x.hour <= 24 else 
                                                                     4)
    if 'weekend' in features:
        # индикатор уикэнда в день начала сессии
        df['weekend'] = df['time1'].apply(lambda x: 1 if x.weekday() in [5, 6] else 0)
        
#     X_new_sparse = csr_matrix(df[features].values)
    encoder = OneHotEncoder()
    X_new_sparse = encoder.fit_transform(df[features].values)
    
    df_sites = df_sites.fillna(0).astype(int)
#     train_sparse = csr_matrix((np.ones(df_sites.values.size), 
#                                 df_sites.values.ravel(), 
#                                 np.arange(df_sites.values.shape[0] + 1) * 
#                                 df_sites.values.shape[1]), dtype=int)[:, 1:]

    # готовим блок с сайтами для кодирования посредством TfidfVectorizer()
    df_sites = df_sites.astype(str)
    sites_text = df_sites['site1']
    for i in range(2, 11):
        sites_text += ' ' + df_sites['site{}'.format(i)]
    # кодируем подготовленный блок с сайтами
    Tfidf_Vectorizer = TfidfVectorizer()    
    train_sparse = Tfidf_Vectorizer.fit_transform(sites_text.values)
   
    X_sparse = sparse.hstack([train_sparse, X_new_sparse]).tocsr()
    X_train_sparse = X_sparse[:train_df.shape[0]]
    X_test_sparse = X_sparse[train_df.shape[0]:]
    
    y_train = df['target'].values[:train_df.shape[0]]
    
    return (X_train_sparse, y_train) if test_df is None else (X_train_sparse, y_train, X_test_sparse)

In [388]:
%%time
X_train_sparse_ohe_tfidf, y_train_ohe, X_test_sparse_ohe_tfidf = feature_engineering_df_OHE_Tfidf(train_df, test_df)
estimator = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
auc = CV_score(estimator, X_train_sparse_ohe_tfidf, y_train_ohe, cv=skf)
print(auc)

0.97748485306
Wall time: 3min 31s


Особого выигрыша не дало, но тем не менее попробуем на **Kaggle**:

In [389]:
%%time
estimator = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
test_pred_proba = estimator.fit(X_train_sparse_ohe_tfidf, y_train_ohe).predict_proba(X_test_sparse_ohe_tfidf)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_sgd_allfeatures_OHE_bal_tfidf.csv'))

Wall time: 1.65 s


**Kaggle: 0.94773**   
Отлично! Результат и позицию еще хоть и немного, но все же **улучшили!  ** 

## <center> 5.6. Cмешивание алгоритмов
Будем делать это по схеме **блендинга**, которая представляет из себя разновидность **стекинга**, описаную в <a href='https://alexanderdyakonov.wordpress.com/2017/03/10/c%D1%82%D0%B5%D0%BA%D0%B8%D0%BD%D0%B3-stacking-%D0%B8-%D0%B1%D0%BB%D0%B5%D0%BD%D0%B4%D0%B8%D0%BD%D0%B3-blending/'>статье</a> на сайте **Александра Дьяконова**.   
   
А именно:  
- делим обучающую выборку на две части
- на первой настраиваем два алгоритма и делаем предсказания на второй
- получаем два метапризнака
- таким же образом получаем два метапризнака на тестовой выборке
- далее, на полученных метапризнаках из второй части обучающей выборки настраиваем линейный классификатор и делаем предсказания на тестовых метапризнаках

In [22]:
import xgboost as xgb

In [19]:
%%time
X_train_sparse_ohe_tfidf, y_train_ohe, X_test_sparse_ohe_tfidf = feature_engineering_df_OHE_Tfidf(train_df, test_df)

Wall time: 7min


In [32]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse_ohe_tfidf, y_train_ohe, 
                                                      test_size=0.5, 
                                                      random_state=17, stratify=y_train_ohe)

In [61]:
%%time
sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
sgd_pred_proba = sgd_logit.fit(X_train, y_train).predict_proba(X_valid)[:, 1]

Wall time: 377 ms


Сразу попробуем **большую** глубину деревьев.

In [62]:
%%time
# инициализируем параметры
params = {
    'objective': 'binary:logistic',
    'max_depth': 11,
    'learning_rate': 0.5,
    'silent': 1.0,
    'n_estimators': 300
}

xgb_classifier = xgb.XGBClassifier(n_jobs=-1, random_state=17, **params)
xgb_pred_proba = xgb_classifier.fit(X_train, y_train).predict_proba(X_valid)[:, 1]

Wall time: 1min 18s


Делаем новые **метапризнаки** и обучаем на них **SGDClassifier**

In [63]:
X_metafeature = np.hstack([sgd_pred_proba.reshape((-1, 1)), xgb_pred_proba.reshape((-1, 1))])

In [64]:
%%time
X_metafeature_test = np.hstack([sgd_logit.fit(X_train, y_train).predict_proba(X_test_sparse_ohe_tfidf)[:, 1].reshape((-1, 1)), 
                            xgb_classifier.fit(X_train, y_train).predict_proba(X_test_sparse_ohe_tfidf)[:, 1].reshape((-1, 1))])

Wall time: 1min 12s


In [66]:
%%time
meta_sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
test_pred_proba = meta_sgd_logit.fit(X_metafeature, y_valid).predict_proba(X_metafeature_test)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, '[YDF & MIPT]_Coursera_allfeatures_OHE_bal_tfidf_meta.csv'))

Wall time: 901 ms


**Kaggle: 0.94406**    
Результат достаточно высокий, но ниже максимального, полученного ранее.

Попробуем **другую комбинацию** использования данных на аналогичном **блендинге**.    
Для компактности соберем все в **одну ячейку** и организуем **измерение времени** отдельных этапов.

Поскольку для градиентного бустинга свойственно **переобучение**, попробуем **уменьшить максимальную глубину** деревьев в **xgboost**.

In [67]:
import datetime

In [70]:
start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
sgd_pred_proba = sgd_logit.fit(X_train, y_train).predict_proba(X_valid)[:, 1]

print('Time for sgd_pred_proba:', datetime.datetime.now() - start_time) # время этапа
start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

# инициализируем параметры
params = {
    'objective': 'binary:logistic',
    'max_depth': 5,
    'learning_rate': 0.5,
    'silent': 1.0,
    'n_estimators': 170
}

xgb_classifier = xgb.XGBClassifier(n_jobs=-1, random_state=17, **params)
xgb_pred_proba = xgb_classifier.fit(X_train, y_train).predict_proba(X_valid)[:, 1]
print('Time for xgb_pred_proba:', datetime.datetime.now() - start_time) # время этапа

X_metafeature = np.hstack([sgd_pred_proba.reshape((-1, 1)), xgb_pred_proba.reshape((-1, 1))])

start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

# вычисляем метапризнаки на тестовой выборке
X_metafeature_test = np.hstack([sgd_logit.fit(X_train_sparse_ohe_tfidf, 
                                              y_train_ohe).predict_proba(X_test_sparse_ohe_tfidf)[:, 1].reshape((-1, 1)), 
                                xgb_classifier.fit(X_train_sparse_ohe_tfidf, 
                                                   y_train_ohe).predict_proba(X_test_sparse_ohe_tfidf)[:, 1].reshape((-1, 1))])

print('Time for X_metafeature_test:', datetime.datetime.now() - start_time) # время этапа
start_time = datetime.datetime.now() # фиксируем время для измерения времени этапа

meta_sgd_logit = SGDClassifier(loss='log', n_jobs=-1, class_weight='balanced', random_state=17)
test_pred_proba = meta_sgd_logit.fit(X_metafeature, y_valid).predict_proba(X_metafeature_test)[:, 1]
    
write_to_submission_file(test_pred_proba, os.path.join(PATH_TO_DATA, 
                                                       '[YDF & MIPT]_Coursera_allfeatures_OHE_bal_tfidf_meta2.csv'))
print('Time for write_to_submission_file:', datetime.datetime.now() - start_time) # время этапа

Time for sgd_pred_proba: 0:00:00.343441
Time for xgb_pred_proba: 0:00:25.080256
Time for X_metafeature_test: 0:00:43.124767
Time for write_to_submission_file: 0:00:00.929278


**Kaggle: 0.94739**    
Неплохо, как мы и ожидали, результат **вырос** при **уменьшении глубины** деревьев, но все же по-прежнему немного **уступает** самому лучшему результату, полученному выше на **линейном** классификаторе **SGDCclassifier**.   
Тем не менее опыт интересный.    
Как известно, **xgboost** любит хорошую настройку. Возможно, при более тщательном подборе гиперпараметров в **xgboost**, например, с использованием библиотеки **hyperopt**, мы смогли бы добиться более высокого результата.

**РЕЗЮМЕ:**   
Итак, финальный результат этой этой части проекта - **результат в [соревновании на Kaggle](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard): 0.94773**.   
На момент его получени это - **63-я** позиция на **Public Leaderboard**, команда в соревновании - **[YDF & MIPT] Dmitry Shereshevskiy**.   

По результатам работы видно, что:
- корректная **предобработка категориальных признаков** дает существенный прирост результата предсказания, и чем таких признаков **больше** - тем лучше
- балансировка классов при их дисбалансе - **важна**
- в этой задаче **линейные методы** показали результат лучше, чем деревья
- можно **смешивать** разные алгоритмы и это тоже дает хороший результат